In [1]:
import numpy
import os
import pickle
from sklearn.cluster.k_means_ import KMeans
import sys
import pandas as pd
from itertools import permutations
import time
# Generate k-means features for videos; each video is represented by a single vector

In [2]:
def distance(x1, x2):
    # x1 and x2 are numpy arrays
    return np.linalg.norm(x1-x2, axis=0)

def closest_center(x):
    # x is a numpy array representing coordinates
    closest_center = 0
    closest_dist = distance(kmeans.cluster_centers_[0], x)
    for i, center in enumerate(np.around(kmeans.cluster_centers_)):
        dist = distance(center, x)
        if dist < closest_dist:
            closest_center = i
            closest_dist = dist
    return closest_center

In [ ]:
if __name__ == '__main__':
    if len(sys.argv) != 4:
        print "Usage: {0} kmeans_model, cluster_num, file_list".format(sys.argv[0])
        print "kmeans_model -- path to the kmeans model"
        print "cluster_num -- number of cluster"
        print "file_list -- the list of videos"
        exit(1)

    kmeans_model = sys.argv[1]; file_list = sys.argv[3]
    cluster_num = int(sys.argv[2])
    mfcc_file_prefix = "../mfcc/"
    mfcc_file_suffix = ".mfcc.csv"
    vector_file = "train_video_vectors.txt"

    # load the kmeans model
    kmeans = pickle.load(open(kmeans_model,"rb"))
    
    with open(file_list, "r") as f:
        video_list = f.read().split("\n")

    video_list = video_list[:-1]
    video_vectors = np.zeros([len(video_list), len(kmeans.cluster_centers_)])
    
    for i, video in enumerate(video_list):
        try:
            df = pd.read_csv(mfcc_file_prefix + video + mfcc_file_suffix, delimiter=';', header=None)
        except IOError:
            print("File Not Found: " + mfcc_file_prefix + video + mfcc_file_suffix)
            continue
        print("File: " + video)
        start = time.time()
        for mfcc in df.values:
            closest_center_index = closest_center(np.around(mfcc))
            video_vectors[i][closest_center_index] += 1
        print(i, time.time() - start)
    np.savetxt(vector_file, video_vectors)
    print(video_vectors)

    print "K-means features generated successfully!"